<a href="https://colab.research.google.com/github/batyark/chest-x-ray/blob/main/project_Xray_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

שימוש בסיפריות המתאימות

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import matplotlib.pyplot as plt
from keras.layers import Dropout
import cv2
from keras.layers import Dense
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report
from keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))



# importing libraries
import glob
%matplotlib inline

#bokeh
from bokeh.models import ColumnDataSource, HoverTool, Panel, FactorRange
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.palettes import Spectral6

upload the files via the google drive

In [ ]:
!unzip /content/drive/MyDrive/chest_xray.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from glob import glob
base_dir = "/content/chest_xray/"
train_dir = os.path.join(base_dir, "train/")
test_dir = os.path.join(base_dir, "test/")
val_dir = os.path.join(base_dir, "val/")
print("Number of images in Train is {}".format(len(glob(train_dir + "*/*"))))
print("Number of images in Test is {}".format(len(glob(test_dir + "*/*"))))
print("Number of images in Validation is {}".format(len(glob(val_dir + "*/*"))))
train_dir_NORMAL = os.path.join(base_dir, "train/NORMAL")
print("Number of images in Train NORMAL is {}".format(len(glob(train_dir_NORMAL + "*/*"))))
train_dir_PNEUMONIA = os.path.join(base_dir, "train/PNEUMONIA")
print("Number of images in Train PNEUMONIA is {}".format(len(glob(train_dir_PNEUMONIA + "*/*"))))
test_dir_NORMAL = os.path.join(base_dir, "test/NORMAL")
print("Number of images in test NORMAL is {}".format(len(glob(test_dir_NORMAL + "*/*"))))
test_dir_PNEUMONIA = os.path.join(base_dir, "test/PNEUMONIA")
print("Number of images in test PNEUMONIA is {}".format(len(glob(test_dir_PNEUMONIA + "*/*"))))
val_dir_NORMAL = os.path.join(base_dir, "val/NORMAL")
print("Number of images in val NORMAL is {}".format(len(glob(val_dir_NORMAL + "*/*"))))
val_dir_PNEUMONIA = os.path.join(base_dir, "val/PNEUMONIA")
print("Number of images in val PNEUMONIA is {}".format(len(glob(val_dir_PNEUMONIA + "*/*"))))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = [[1351, 234, 8],
[3875, 390, 8]]
X = np.arange(3)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, data[1], color = 'r', width = 0.25)
ax.set_ylabel('Number of images')
ax.legend(labels=['NORMAL', 'PNEUMONIA'])

In [ ]:
import glob
main_path = "../input/labeled-chest-xray-images/chest_xray"

train_dir = os.path.join(base_dir, "train/")
test_dir = os.path.join(base_dir, "test/")


train_normal = glob.glob(train_dir+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_dir+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_dir+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_dir+"/PNEUMONIA/*.jpeg")
IMG_SIZE = 224
BATCH = 16
SEED = 42

In [ ]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

חלוקה לוולידציה

In [ ]:
train_df, val_df = train_test_split(df_train, test_size = 0.15, random_state = SEED, stratify = df_train['class'])


In [ ]:
occur = train_df.groupby(['class']).size()
display(occur)
COUNT_PNEUMONIA=3293
COUNT_NORMAL=1140

initial_bias = np.log([COUNT_PNEUMONIA / COUNT_NORMAL])
print("Initial bias: {:.5f}".format(initial_bias[0]))

TRAIN_IMG_COUNT = COUNT_NORMAL + COUNT_PNEUMONIA
weight_for_0 = (1 / COUNT_NORMAL) * (TRAIN_IMG_COUNT) / 2.0
weight_for_1 = (1 / COUNT_PNEUMONIA) * (TRAIN_IMG_COUNT) / 2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print("Weight for class 0: {:.2f}".format(weight_for_0))
print("Weight for class 1: {:.2f}".format(weight_for_1))


In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.)
val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,
                                             #directory=train_path, #dataframe contains the full paths
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,
                                            #directory=train_path,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,
                                            #directory=test_path,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)

קריאה לסיפריות נוספות

In [ ]:
from keras.layers import Flatten,Dense,BatchNormalization,Dropout,LeakyReLU,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import ResNet50V2
from keras import Sequential

שימוש בRESENET50

In [ ]:
resnet = ResNet50V2(weights="imagenet",input_shape=(224,224,3),include_top=True)
for layer in resnet.layers[:191]:
    layer.trainable = False
for layer in resnet.layers[191:]:
    layer.trainable = True
model = Sequential(name="my_sequential")
model.add(resnet)
model.add(Flatten(input_shape=[224,224,1]))
model.add(Dense(1,activation="sigmoid"))
optim = Adam(lr=1e-3)
model.compile(optimizer=optim,loss='binary_crossentropy',metrics=["accuracy"])
model.summary()

densenet שימוש *ב*

In [ ]:
densenet =tf.keras.applications.densenet.DenseNet121(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=(224,224,3), pooling=None, classes=1000
)
for layer in densenet.layers:
  layer.trainable = False

model = Sequential(name="my_sequential")

model.add(densenet)
model.add(Flatten(input_shape=[224,224,1]))
model.add(Dense(1,activation="sigmoid"))
optim = Adam(lr = 1e-2)
model.compile(optimizer=optim,loss='binary_crossentropy',metrics=["accuracy"])
model.summary()


אימון המודל

In [ ]:
history = model.fit(ds_train,batch_size = 16, epochs = 100,validation_data=ds_val,steps_per_epoch=16,validation_steps=16,class_weight=class_weight,
    callbacks=[checkpoint_cb, early_stopping_cb])


In [ ]:
plt.plot(history.history['accuracy']) 
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy') 
plt.ylabel('accuracy') 
plt.xlabel('epoch') 
plt.legend(['train', 'validation'], loc='lower right') 
plt.show()

In [ ]:
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss'])
plt.title('model loss') 
plt.ylabel('loss') 
plt.xlabel('epoch') 
plt.legend(['train', 'validation'], loc='upper right') 
plt.show()

In [ ]:
import seaborn as sns
def predict_classes(predictions):
    valResult = predictions.copy()
    valResult[valResult <= 0.5] = 0  
    valResult[valResult > 0.5] = 1   
    return valResult
def plot_confusion_matrix(predictions, labels):
    valResult = predict_classes(predictions)
    conf_matrix_df= pd.DataFrame(confusion_matrix(labels,valResult),
                                 index=["Normal", "Pneumonia"],
                                 columns=["Normal", "Pneumonia"])
    plt.figure(figsize=(7,6))
    plt.title("Confusion Matrix")
    sns.heatmap(conf_matrix_df, 
                annot=True, 
                annot_kws={"size" : "20"}, 
                fmt='.4g')
    plt.xlabel('\nPredicted', 
               fontsize=12)
    plt.ylabel('Actual', 
               fontsize=12, 
               rotation='horizontal', 
               labelpad=40)
    return cm

In [ ]:
prediction = model.predict(ds_test)
actual_class = ds_test.classes
# Confusion Matrix
valResult = predict_classes(prediction)
cm = confusion_matrix(actual_class, valResult) 
print('='*50)                                
print('True Positive  (TP) = ', cm[0][0])
print('False Positive (FP) = ', cm[0][1])
print('False Negative (FN) = ', cm[1][0])
print('True Negative  (TN) = ', cm[1][1])
print('-'*26)
plot_confusion_matrix(prediction, actual_class)

if (cm[0][0] + cm[0][1])!=0 and (cm[0][0], cm[1][0])!=0:
      precision= round((cm[0][0] / (cm[0][0] + cm[0][1]))*100, 2) 
      recall   = round((cm[0][0] / (cm[0][0] + cm[1][0]))*100, 2)     
      print(f'Precision Score: {precision}%')
      print(f'Recall Score: {recall}%')
      print('='*50) 
      print('\n')